<a href="https://colab.research.google.com/github/profesorKiki/Ejemplos_Colab/blob/main/EjemplosDeCreaci%C3%B3nDeProcesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica de procesos 

Ejemplo en **Python**

In [27]:
%%writefile fork.py 
import sys
import os
import time
	
def parent():    
    for i in range(2):    
      pid = os.fork()
      if pid < 0:
          print("Error al crear el nuevo proceso")
          os._exit(1)
      
      if pid == 0:
          print(f"Hijo {i+1} ({ os.getpid() }) - Mi padre es ({ os.getppid() })" )
          time.sleep(10)
          sys.stdout.flush() #Para forzar la salida de los hijos cuando se redirige al archivo
          os._exit(0)
    
    print(f"Proceso padre ({ os.getpid() })" )
    for _ in range(2):    
      print( os.wait() )
      
parent()

Overwriting fork.py


In [28]:
!chmod 700 ./fork.py

In [29]:
!nohup python fork.py 1>salida 2>/dev/null &

In [30]:
!pstree -pc 

docker-init(1)─┬─node(8)─┬─colab-fileshim.(61)
               │         ├─dap_multiplexer(83)─┬─{dap_multiplexer}(84)
               │         │                     ├─{dap_multiplexer}(85)
               │         │                     ├─{dap_multiplexer}(86)
               │         │                     └─{dap_multiplexer}(87)
               │         ├─jupyter-noteboo(82)─┬─python3(54836)─┬─pstree(61451)
               │         │                     │                ├─{python3}(54841)
               │         │                     │                ├─{python3}(54842)
               │         │                     │                ├─{python3}(54843)
               │         │                     │                ├─{python3}(54844)
               │         │                     │                ├─{python3}(54845)
               │         │                     │                ├─{python3}(54846)
               │         │                     │                ├─{python3}(54848)
        

Ejemplo en **C**

In [38]:
%%writefile fork.c
#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>

int main(int argc, char *argv[])
{
    pid_t pid;
    pid = fork();
    
    if( pid < 0 )
    {
        printf("Error al crear el nuevo proceso\n");
        return EXIT_FAILURE;
    }
    
    if( pid )
    {
        printf("Soy el proceso padre. PID: %d\n",getpid());
        wait(NULL);
    }
    else
    {
        printf("Soy el proceso hijo. PID: %d\n",getpid());
        printf("Mi padre es el proceso con PID: %d\n",getppid());
        sleep(10);
        return EXIT_SUCCESS;	
    }

    return EXIT_SUCCESS;
}

Overwriting fork.c


In [39]:
!gcc -o fork.bin fork.c 

In [46]:
!nohup ./fork.bin 1>salida 2>/dev/null & 

In [47]:
!pstree -pc

docker-init(1)─┬─fork.bin(63945)───fork.bin(63946)
               ├─node(8)─┬─colab-fileshim.(61)
               │         ├─dap_multiplexer(83)─┬─{dap_multiplexer}(84)
               │         │                     ├─{dap_multiplexer}(85)
               │         │                     ├─{dap_multiplexer}(86)
               │         │                     └─{dap_multiplexer}(87)
               │         ├─jupyter-noteboo(82)─┬─python3(54836)─┬─pstree(63957)
               │         │                     │                ├─{python3}(54841)
               │         │                     │                ├─{python3}(54842)
               │         │                     │                ├─{python3}(54843)
               │         │                     │                ├─{python3}(54844)
               │         │                     │                ├─{python3}(54845)
               │         │                     │                ├─{python3}(54846)
               │         │              

Ejemplo en **Java**

In [61]:
%%writefile crear_proceso.java
import java.io.IOException;

public class Main
{
    public static void main(String[] args) throws IOException,  InterruptedException
    {
        ProcessBuilder pb = new ProcessBuilder("java", "Hijo.java");
        pb.redirectErrorStream(true);
        pb.inheritIO();
        
        Process proceso = pb.start();
        
        int error = proceso.waitFor();
        
        if( error != 0 )
        {
            System.out.println("Error al ejecutar el programa");
        }
    }
}

Overwriting crear_proceso.java


In [68]:
%%writefile Hijo.java
class Hijo 
{
    public static void main(String[] args) throws InterruptedException
    {
        ProcessHandle yo = ProcessHandle.current();
        long pid  = yo.pid();
        long ppid = yo.parent().get().pid();
        System.out.println("Soy el proceso: " + pid + " y mi padre es: " + ppid );
        Thread.sleep(10000);
    }
}

Overwriting Hijo.java


In [75]:
!nohup java crear_proceso.java 1>salida 2>/dev/null &  

In [76]:
!pstree -pT

docker-init(1)─┬─java(72528)───java(72550)
               ├─node(8)─┬─colab-fileshim.(61)
               │         ├─dap_multiplexer(83)
               │         ├─jupyter-noteboo(82)───python3(54836)───pstree(72578)
               │         ├─node(56420)
               │         ├─oom_monitor.sh(30)───sleep(72577)
               │         └─python3(60)
               ├─python3(189)
               ├─python3(54796)
               ├─python3(54799)
               ├─python3(56389)
               └─run.sh(32)───kernel_manager_(34)
